In [ ]:
%reload_ext autoreload
%autoreload 2
import os
import sys
import re
from collections import Counter, defaultdict
import json
from tqdm.notebook import tqdm
from datetime import datetime
# import republic.model.republic_document_model as rdm
import pandas as pd
from substitutions import substitutions
from deletions import deletions
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
repo_dir = os.path.split(os.getcwd())[0]
if repo_dir not in sys.path:
    sys.path.append(repo_dir)



## Stap 1: Aanmaken en opschonen DataFrame

In deze eerste stap creëren we een DataFrame met de annotaties. Dit DataFrame bevat:

- Geaggregeerde verwijzingen: Alle verwijzingen zijn samengevoegd in één DataFrame. Met verwijzingen bedoelen we de output van de NER-tagger, dus de entiteiten zoals ze in de tekst zijn herkend.
- Annotatiekolommen: De kolommen `anno_name1` tot en met `anno_name5` zijn toegevoegd om annotaties (standaardversies van achternamen) te registreren.
- Subframes: We hebben regels opgesteld om subframes te herkennen, zoals `df_delete` en `df_unmatched`, om te kunnen meten hoeveel rijen er al wel of niet thuisgebracht zijn.

Met de stap `df = df[df['aantal'] >= 10]` kunnen we kiezen om te werken met het hele DataFrame, inclusief strings die maar één keer voorkomen, of om te beginnen met vaker voorkomende strings.


Hier filter ik de strings van "annotaties_danig" eruit, dit zijn namelijk de annotaties die alleen maar hoedanigheden bevatten en geen personen. Dit is afkomstig van het notebook dat hoedanigheden verwerkt.

In [ ]:
#load annotations minus hoedanigheden
annotations = pd.read_csv('annotations-layer_PER.tsv', sep='\t')
annotations_danig = pd.read_csv('../HOE/matched_hoedanigheden.tsv', sep='\t')
merged_df = pd.merge(
    annotations,
    annotations_danig[['resolution_id', 'paragraph_id', 'offset', 'end', 'matched']],
    on=['resolution_id', 'paragraph_id', 'offset', 'end'],)

# Filter out rows where annotations_danig['matched'] has an 'x'
rows_to_drop = merged_df[merged_df['matched']]
annotations = annotations[~annotations.index.isin(rows_to_drop.index)]
print(f'{len(rows_to_drop)} coincide exactly with an attribution; dropped')

In [ ]:
annotations['tag_new'] = annotations['tag_text'].apply(lambda x: re.sub(r'^[^a-zA-Z0-9]+', '', x))

df_temp = annotations.groupby('tag_new').size().reset_index(name='aantal')
column_names = ['tag', 'lowertag', 'aantal', 'anno_name1', 'anno_name2', 'anno_name3',
                'anno_name4', 'anno_name5', 'delete', 'best_score', 'select', 'uitzoeken/onduidelijk']
df = pd.DataFrame(columns=column_names)
df['tag'] = df_temp['tag_new']
df['aantal'] = df_temp['aantal']
print(f"Dataframe bestaat uit {len(df)} rijen")
df = df[df['aantal'] >= 10] #We doen het nu even meet alles wat meer dan 20 keer voorkomt, om sneller overal doorheen te lopen
df['lowertag'] = df['tag'].str.lower()
anno_columns = ['anno_name1', 'anno_name2', 'anno_name3', 'anno_name4', 'anno_name5']
df_delete = df[df['delete'] == 'x']
df_zonder_delete = df[~(df['delete'] == 'x')]
df_matched = df_zonder_delete[df_zonder_delete[['anno_name1', 'anno_name2', 'anno_name3', 'anno_name4']].notna().any(axis=1)]
df_unmatched = df_zonder_delete[df_zonder_delete[anno_columns].isna().all(axis=1) & ~df['delete'].astype(str).eq('x')]


for column in anno_columns:
    if df[column].dtype != 'object':  
        df[column] = df[column].astype(str)

We maken een gestandaardiseerde versie van de verwijzingen aan, waarbij spellingsnormalisatie is toegepast. Dit passen we alleen toe op de "lowertag", dat wil zeggen, een kopie van de string die in kleine letters is. De oorspronkelijke strings blijven staan. Door alles in kleine letters te zetten, zijn de tags niet hoofdlettergevoelig. Dit omdat de namen niet altijd met hoofdletters geschreven of herkend zijn.

De vervangingen in `substitutions.py` zijn grofweg onder te verdelen in:
- leestekens: standaardiseren van komma's, spaties, en verwijderen van onnodige leestekens
- Veelvoorkomende woorden: standaardiseren van woorden als "admiraal", "ambassadeur", "baron", "keurvorst" e.d. om (delen van) strings makkelijker weg te kunnen zetten als hoedanigheden (en dus geen personen)
- Spelling: Om spellingsvariatie te verminderen en matchen makkelijker te maken, vervangen we "ae" door "aa", "gt" door "cht" en "uij" of "uy" door "ui".
- We hebben ook de verschillende verwijzingen naar afkomst, geboorteplaats en woonplaats gestandaardiseerd, zo wordt bijvoorbeeld "inwoonder van" en "woonende te" allebei "woonachtig in". 

In [ ]:
import re
from tqdm import tqdm
from substitutions import substitutions

def apply_substitutions(df):
    # Apply substitutions to DataFrame
    substitution_count = 0
    for pattern, replacement in tqdm(substitutions.items(), desc="Applying substitutions to DataFrame"):
        try:
            temp = df['lowertag'].str.count(pattern, flags=re.IGNORECASE)
            substitution_count += temp.sum()
            df['lowertag'] = df['lowertag'].str.replace(pattern, replacement, regex=True)
        except Exception as e:
            print(f"Error occurred with pattern: {pattern}")
            print(e)

    print(f"Done! {substitution_count} substitutions made in DataFrame")

    return df

# Example usage
df = apply_substitutions(df)

df.loc[df['lowertag'] == '', 'delete'] = 'x'
df_delete = df[df['delete'] == 'x']
df_zonder_delete = df[~(df['delete'] == 'x')]

anno_columns = ['anno_name1', 'anno_name2', 'anno_name3', 'anno_name4']
df_matched = df_zonder_delete[df_zonder_delete[anno_columns].notna().any(axis=1)]
df_unmatched = df_zonder_delete[df_zonder_delete[anno_columns].isna().all(axis=1) & ~df['delete'].astype(str).eq('x')]


In [ ]:
df.to_pickle('fulldfsubs.pkl')

## Stap 2: Koppelen persoonsnamen

De belangrijkste stap in dit notebook is de koppeling: het toepassen van de thesaurus van persoonsnamen op de opgeschoonde strings (`lowertag`). De thesaurus bestaat uit reguliere expressies. De thesaurus is bottom-up opgebouwd uit het materiaal zelf. We ontdekten dat persoonsnamen vaak worden ingeleid door hoedanigheids-woorden, zoals "admiraal" of "lieutenant", of "heer" of "heeren". Dit iteratieve proces resulteerde in het kws_dict dictionary, dat bestaat uit reguliere expressies en hun bijbehorende tags, bijvoorbeeld: `"ro+[ijy]+en": "rooyen"`. Sommige expressies zijn automatisch gegenereerd door algemene regels, zoals het vervangen van de letters u en v door [uv], omdat deze vaak door elkaar worden gebruikt of verkeerd worden herkend. De thesaurus is verder aangevuld met namen uit eerdere corpora, zoals het Repertorium Ambtenaren en Ambtsdragers en Schutte.
Door iteratief te werk te gaan, hebben we de expressies steeds verder verfijnd en opgeschoond. Met behulp van Levenshtein-afstand konden we snel zien welke expressies naar dezelfde naam verwezen en deze samenvoegen. Door de output te sorteren op frequentie, konden we gemakkelijk ruis in de vorm van ongewenste tags, zoals `heer van`, identificeren en verwijderen.

De koppeling houdt in dat we standaardversies van namen toekennen aan verwijzingen uit de tagger, waarbij de opgeschoonde versie als tussenstap dient. Bijvoorbeeld, `Advocaet Fiscael Bilderbeecq` wordt opgeschoond tot `advocaat fiscaal bilderbeecq`. Dit wordt gematcht met het patroon `bilderb\w*` en krijgt de annotatie `bilderbeek`.

We pasten deze reguliere expressies steeds toe op df_unmatched['lowertag']. Bij een match wordt de tag toegevoegd aan de eerstvolgende beschikbare anno_name kolom, beginnend met `anno_name1`, waardoor de rij wordt toegevoegd aan `df_matched`. Het doel is om bij iedere iteratie de thesaurus uit te breiden en het aantal niet-gematchte verwijzingen te verminderen.

In deze stap passen we annotaties toe op de opgeschoonde gegevens met behulp van reguliere expressies uit `kws_hoe`. Het doel is om relevante tags te identificeren en aan de data toe te voegen.

We proberen elk regex-patroon uit de patterns-lijst op de lowertag toe te passen. Als een patroon een match vindt, wordt de string opgedeeld in drie delen: vóór de match [0], de match zelf [1], en na de match [2]. Deze delen worden samen met de overeenkomstige tag en het originele patroon toegevoegd aan een tijdelijk resultaat DataFrame [`new_df`]. We bewaren het originele regex patroon in de kolom `provenance`. 

In [ ]:
def apply_annotations(df, kw_dict):
    patterns = []

    # Compile regex patterns from the dictionary
    for pat, repl in kw_dict.items():
        try:
            pattern_with_group = f"(.*) *(\\b{pat}\\b) *(.*)"
            ptrn = re.compile(pattern_with_group, flags=re.I)
            patterns.append((ptrn, pat, repl))  # Store the pattern, original 'pat', and replacement
        except re.error as e:
            print(f"Error met het patroon '{pattern_with_group}': {e}")

    results = []

    # Iterate over each row in the DataFrame with a progress bar
    for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
        lowertag = row['lowertag']
        aantal = row['aantal']  # Preserve the 'aantal' value
        matched = False

        for pattern, pat, replacement in patterns:
            if matched:
                break

            match = pattern.match(lowertag)
            if match:
                matched = True
                matches = pd.DataFrame({
                    0: [match.group(1)],
                    1: [match.group(2)],
                    2: [match.group(3)],
                    'provenance': [pat],  # Use the original 'pat' for provenance
                    'anno_name1': [replacement],  # Use the 'replacement' from kw_dict
                    'aantal': [aantal]  # Add the 'aantal' column to the result
                }, index=[idx])
                results.append(matches)

    if results:
        offpat = pd.concat(results)
        offpat = offpat.sort_values('provenance')
    else:
        offpat = pd.DataFrame(columns=[0, 1, 2, 'provenance', 'anno_name1', 'aantal'])

    return offpat


In [ ]:
from shortdict3 import kws_dict
offpat = apply_annotations(df_unmatched, kws_dict)
new_df = offpat
print(f"gematcht: {len(new_df)}")


In [ ]:
#new_df.to_pickle('offpataug20.pkl')
#new_df.to_csv('offpat20aug.tsv', sep='\t')

Om de evaluatie te vereenvoudigen, bekijken we elke unieke herkende entiteit-string slechts één keer; dit frame noemen we `preview`. Op deze manier kunnen we snel controleren welke gegevens door de reguliere expressies worden opgevangen, zonder dat het dataframe te lang en onoverzichtelijk wordt.

In [ ]:
preview = new_df.groupby(1).first().reset_index() #bekijk elke unieke string maar 1x
preview = preview[new_df.columns]
preview.to_csv('previewframe.tsv',sep='\t')
preview.head(100)

## Stap 3a: resultatenframe "0" doorzoeken, dus de onderdelen van de entiteiten vóór de herkende string.

In [ ]:
def mark_pat(df, field, kw_dict, target_name):
    results = []

    # Compile all patterns beforehand
    compiled_patterns = {}
    for pattern, replacement in kw_dict.items():
        pattern_with_group = f"(.*) *(\\b{pattern}\\b) *(.*)"
        compiled_patterns[pattern] = {
            'compiled_pattern': re.compile(pattern_with_group, flags=re.I),
            'replacement': replacement
        }
    
    # Apply annotations using the compiled patterns
    for pattern, data in compiled_patterns.items():
        ptrn = data['compiled_pattern']
        replacement = data['replacement']
        
        # Perform regex matching
        matches = df[field].str.extractall(ptrn)
        
        if not matches.empty:
            matches['provenance'] = pattern
            matches[target_name] = replacement
            results.append(matches)

    # Return the concatenated DataFrame or an empty DataFrame if no results
    if results:
        offpat = pd.concat(results)
        offpat = offpat.sort_values('provenance')
        
        # Rename columns based on target_name
        if target_name == 'anno_name0':
            offpat = offpat.rename(columns={0: 'prov_0'})
        elif target_name == 'anno_name2':
            offpat = offpat.rename(columns={2: 'prov_2'})
        
        offpat['old_ind'] = offpat.index.get_level_values(0)  # Keep old indices

        return offpat

    # Return an empty DataFrame if no matches
    return pd.DataFrame(columns=[0, 1, 2, 'provenance', target_name])


In [ ]:
import re
import pandas as pd
from tqdm import tqdm

# Set the index and rename columns
new_df.index = [idx[0] if isinstance(idx, tuple) else idx for idx in new_df.index]
new_df = new_df.rename(columns={'provenance': 'prov_1'})

preresults = {}

# Use tqdm for a single progress bar
for pattern, replacement in tqdm(kws_dict.items(), desc="Processing patterns", unit="pattern"):
    target_col = 'anno_0'  # Since there's no distinction, we always target 'anno_0'
    
    # Call mark_pat with the relevant keyword and pattern
    offpat = mark_pat(new_df, field=0, kw_dict={pattern: replacement}, target_name=target_col)

    # Only append results if the DataFrame is not empty
    if not offpat.empty:
        preresults[pattern] = offpat

# Concatenate results if there are any
if preresults:
    preresult_tot = pd.concat(preresults)
    print("Totaal gevonden:", len(preresult_tot))
else:
    preresult_tot = pd.DataFrame(columns=['index', '0'])
    print("Niets gevonden.")

def ensure_dataframe(df, new_col_name):
    if isinstance(df, pd.Series):
        df = df.to_frame()
    if df.empty:
        df = pd.DataFrame(columns=[new_col_name])
    else:
        df.columns = [new_col_name] + df.columns.tolist()[1:]
    return df

preresult_tot = ensure_dataframe(preresult_tot, '0')


Nu voegen we de resultaten van deze zoektocht samen met het resultatenframe (new_df).

In [ ]:
import numpy as np
import pandas as pd

# Step 1: Initialize new columns in new_df
new_df['anno_0'] = np.nan
new_df['prov_0'] = np.nan

# Step 2: Function to merge and concatenate annotations, ensuring no duplicates
def merge_annotations(src_df, target_df, src_col, tgt_col):
    # Create a dictionary to hold sets of aggregated values for uniqueness
    temp_dict = {}
    for idx, row in src_df.iterrows():
        old_ind = row['old_ind']
        value = row[src_col]
        if old_ind in temp_dict:
            temp_dict[old_ind].add(value)
        else:
            temp_dict[old_ind] = {value}

    # Update target_df based on the aggregated dictionary
    for index, values in temp_dict.items():
        new_value = ', '.join(sorted(values))
        if index in target_df.index:
            existing_value = target_df.at[index, tgt_col]
            if pd.isna(existing_value):
                target_df.at[index, tgt_col] = new_value
            else:
                # Combine existing and new values, avoid duplicates
                combined_values = set(existing_value.split(', ')) | values
                target_df.at[index, tgt_col] = ', '.join(sorted(combined_values))

# Step 3: Apply the function to each DataFrame
merge_annotations(preresult_tot, new_df, 'anno_0', 'anno_0')
merge_annotations(preresult_tot, new_df, 'provenance', 'prov_0')

# Printing or returning new_df to see the updates
# new_df


Stap 3b: `2` doorzoeken, dus de onderdelen van de entiteiten ná de herkende string.

In [ ]:
postresults = []

# Use tqdm for a single progress bar and process through field=2
for pattern, replacement in tqdm(kws_dict.items(), desc="Processing postresults", unit="pattern"):
    target_col = 'anno_2'  # Assuming you want to target a different annotation column for postresults
    
    # Call mark_pat with the relevant keyword and pattern
    offpat = mark_pat(new_df, field=2, kw_dict={pattern: replacement}, target_name=target_col)

    # Only append results if the DataFrame is not empty
    if not offpat.empty:
        postresults.append(offpat)

# Concatenate results if there are any
if postresults:
    postresult_tot = pd.concat(postresults, ignore_index=True)
    print("Totaal gevonden voor postresults:", len(postresult_tot))
else:
    postresult_tot = pd.DataFrame(columns=['index', '2'])
    print("Niets gevonden voor postresults.")

def ensure_dataframe(df, new_col_name):
    if isinstance(df, pd.Series):
        df = df.to_frame()
    if df.empty:
        df = pd.DataFrame(columns=[new_col_name])
    else:
        df.columns = [new_col_name] + df.columns.tolist()[1:]
    return df

postresult_tot = ensure_dataframe(postresult_tot, '2')


In [ ]:
import numpy as np
import pandas as pd

# Nieuwe kolommen aanmaken
new_df['anno_2'] = np.nan  
new_df['prov_2'] = np.nan  

# Enhanced function to merge and concatenate annotations, ensuring no duplicates
def merge_annotations(src_df, target_df, src_col, tgt_col):
    # Create a dictionary to hold sets of aggregated values for uniqueness
    temp_dict = {}
    for idx, row in src_df.iterrows():
        old_ind = row['old_ind']
        value = row[src_col]
        if old_ind in temp_dict:
            temp_dict[old_ind].add(value)
        else:
            temp_dict[old_ind] = {value}

    # Update target_df based on the aggregated dictionary
    for index, values in temp_dict.items():
        if index in target_df.index:
            existing_value = target_df.at[index, tgt_col]
            if pd.isna(existing_value):
                # Directly assign if no existing value
                target_df.at[index, tgt_col] = ', '.join(sorted(values))
            else:
                # Split existing values, combine with new, ensure uniqueness, then join
                existing_set = set(existing_value.split(', '))
                combined_values = existing_set | values
                target_df.at[index, tgt_col] = ', '.join(sorted(combined_values))

# Apply the function to each DataFrame
merge_annotations(postresult_tot, new_df, 'anno_2', 'anno_2')
merge_annotations(postresult_tot, new_df, 'provenance', 'prov_2')

# Tussentijds opslaan
new_df.to_pickle('new_df2.pkl')

Het resultatenframe `new_df` is eenvoudig van opzet: het bevat de genormaliseerde strings (`lowertag`), opgesplitst in drie onderdelen (0, 1, 2), en de toegekende annotaties (`anno_name1-5`) en reguliere expressies (`provenance`). In de volgende stap combineren we het geaggregeerde frame `df` met de resultaten uit `new_df`, zodat alle informatie (inclusief de oorspronkelijke tag, aantallen, en nog niet herkende strings) op één centrale plek staat.

In [ ]:
import numpy as np
df['provenance'] = np.nan
df['prov_0'] = np.nan
df['prov_1'] = np.nan
df['prov_2'] = np.nan
df['prov_2'] = np.nan
df['anno_name1'] = new_df['anno_name1'] #where the magic happens
df['anno_name3'] = new_df['anno_0'] #where the magic happens
df['anno_name2'] = new_df['anno_2'] #where the magic happens

df_delete = df[df['delete'] == 'x']
df_zonder_delete = df[~(df['delete'] == 'x')]
df_matched = df_zonder_delete[df_zonder_delete[['anno_name1', 'anno_name2', 'anno_name3', 'anno_name4']].notna().any(axis=1)]
df_unmatched = df_zonder_delete[df_zonder_delete[anno_columns].isna().all(axis=1) & ~df['delete'].astype(str).eq('x')]
df_matched.to_csv('test.tsv',sep='\t')


Optionele tussenstap: bereken welk percentage van de verwijzingen tags heeft gekregen. Dit percentage is relatief aan het vooraf gespecificeerde minimumaantal van voorkomens, in ons geval 10 (zie bovenaan).

In [ ]:
total_entities = df['aantal'].sum()
total_matched_entities = df_matched['aantal'].sum() + df_delete['aantal'].sum()
total_unmatched_entities = df_unmatched['aantal'].sum()

# percentage berekenen
if total_entities > 0:
    percentage_matched = (total_matched_entities / total_entities) * 100
else:
    percentage_matched = 0

print(f"Hoeveelheid matches: {percentage_matched:.2f}%")


We hebben nu een overzicht van alle door de NER-herkende verwijzingen naar persoonsnamen, en bijbehorende tags die hierbij horen. 

Ons doel is om in de Goetgevonden-applicatie de tags direct in de tekst weer te geven. Hiervoor moeten we ervoor zorgen dat elke verwijzing een exacte locatie in de tekst heeft, zodat gebruikers op de tekst kunnen klikken en de bijbehorende tag zien. De oorspronkelijke output van de NER-tagger, waarmee we dit notebook hebben geopend, bevat deze informatie.

Nu willen we de geaggregeerde tag-informatie terugzetten in dit bestand. Het eindresultaat van dit notebook is om verwijzingen naar persoonsnamen te verrijken met standaardnamen. Deze persoonsnamen moeten genest worden weergegeven (dat wil zeggen, met meerdere, soms overlappende persoonsnamen in de tekst waar relevant).

De volgende stappen zijn gericht op het koppelen van de verzamelde informatie aan de oorspronkelijke tekstverwijzingen, inclusief hun exacte locatie in de tekst.

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Ensure that both columns are strings and strip any leading/trailing whitespace
annotations['tag_text'] = annotations['tag_text'].astype(str).str.strip()
df_matched['tag'] = df_matched['tag'].astype(str).str.strip()

# Perform the merge, keeping all columns from both DataFrames
annotations = annotations.merge(df_matched, left_on='tag_text', right_on='tag', how='left', suffixes=('', '_matched'))

# Check if the merge has resulted in NaNs and if there are indeed matching values
if annotations.isna().all().all():
    print("Some columns have only NaN values after merging.")
    # Optional: Debug by checking a few mismatches
    print("Examples of mismatched rows:")
    print(annotations[annotations['tag'].isna()].head())


In [ ]:

annotations = pd.read_csv('annotations-layer_PER.tsv', sep='\t')

# Apply regex to clean 'tag_text' and create 'tag_new' column
annotations['tag_new'] = annotations['tag_text'].str.replace(r'^[^a-zA-Z0-9]+', '', regex=True)

# Define the columns that need to be copied from df_matched to annotations
columns_to_add = [
    'lowertag', 'anno_name1', 'anno_name2', 'anno_name3', 'anno_name4', 'anno_name5',
    'delete',
]

# Ensure all required columns are present in annotations
for col in columns_to_add:
    if col not in annotations.columns:
        annotations[col] = np.nan

# Normalize 'tag_text' in annotations and 'tag' in df_matched
annotations['tag_text'] = annotations['tag_text'].str.strip().str.lower()
df_matched['tag'] = df_matched['tag'].str.strip().str.lower()

# Remove duplicates in df_matched by keeping the first occurrence of each tag
df_matched_unique = df_matched.drop_duplicates(subset='tag', keep='first')

# Merge the two DataFrames to add columns from df_matched to annotations
# Use a left merge to retain all rows from annotations
annotations = annotations.merge(df_matched_unique, left_on='tag_text', right_on='tag', how='left', suffixes=('', '_matched'))

# Rename and reorder columns as needed
for col in columns_to_add:
    if col in annotations.columns:
        annotations[col] = annotations[f'{col}_matched']

# Drop intermediate columns
annotations = annotations.drop(columns=[f'{col}_matched' for col in columns_to_add if f'{col}_matched' in annotations.columns])

# Sort the annotations DataFrame by 'anno_name1'
annotations = annotations.sort_values('anno_name2')

# Display the first 100 rows of the updated annotations DataFrame
annotations.head(100)


In [ ]:
total_rows = len(annotations)

# Calculate the number of rows where 'anno_name1' is not empty or NaN
non_empty_anno_name1 = annotations['anno_name1'].notna() & (annotations['anno_name1'].str.strip() != '')

# Calculate the percentage of rows with non-empty 'anno_name1'
percentage_non_empty = non_empty_anno_name1.sum() / total_rows * 100

# Print the result
print(f"Percentage of annotations with non-empty 'anno_name1': {percentage_non_empty:.2f}%")


Het eindresultaat van dit notebook willen we exporteren als JSON, waarin de tekstverwijzingen samen worden gevoegd met de standaardversies van persoonsnamen.

In [ ]:
import pandas as pd
import json
from tqdm import tqdm

# Columns for person names
anno_columns = ['anno_name1', 'anno_name2', 'anno_name3', 'anno_name4', 'anno_name5']

# Filter annotations that have at least one non-null value in the anno_columns
annotations_matched = annotations[~annotations[anno_columns].isna().all(axis=1)]

# Counter to limit printing
print_counter = 0
max_prints = 10

def create_json_objects(row):
    global print_counter
    json_objects = []
    
    for col in anno_columns:
        if pd.notnull(row[col]):
            # Split the column values by commas
            entities = [entity.strip() for entity in str(row[col]).split(',')]

            # Print only if there are multiple entities and we haven't exceeded the print limit
            if len(entities) > 1 and print_counter < max_prints:
                print_counter += 1
                print(f"Found multiple persoonsnamen: {', '.join(entities)}")

            # Iterate through each entity
            for entity_name in entities:
                provenance = None

                # Handle provenance based on the entity column
                if col == 'anno_name1':
                    provenance = row['prov_1']
                elif col == 'anno_name2':
                    provenance = row['prov_2']
                elif col == 'anno_name3':
                    provenance = row['prov_0']

                json_object = {
                    "entity": {
                        "name": entity_name,
                        "category": "PER",
                        "labels": []  # No categories provided, keeping an empty list
                    },
                    "reference": {
                        "layer": row['layer'],
                        "inv": row['inv'],
                        "tag_text": row['tag_text'],
                        "resolution_id": row['resolution_id'],
                        "paragraph_id": row['paragraph_id'],
                        "offset": row['offset'],
                        "end": row['end'],
                        "tag_length": row['tag_length']
                    }
                }

                if provenance and provenance == provenance:
                    json_object['reference']['provenance'] = [ provenance ]

                json_objects.append(json_object)
    
    return json_objects

# Process each row to create JSON objects
json_data_list_of_lists = []
for index, row in tqdm(annotations_matched.iterrows(), total=len(annotations_matched), desc='Creating JSON objects'):
    json_data_list_of_lists.extend(create_json_objects(row))

# Write the JSON objects to a file
with open('annotations_matched_simple.json', 'w') as json_file:
    json.dump(json_data_list_of_lists, json_file, indent=4)

print('annotations_matched_simple.json')


In [ ]:
len(json_data_list_of_lists)